In [1]:
import atoti as tt

In [2]:
session = tt.Session()

In [3]:
# Make sure the csv file is in the correct format.

table = session.read_csv(
    "date-filter-use-case.csv",
    table_name="date-filter-use-case",
    separator=",",
    keys=["DATE"],
    types={"DATE": tt.type.LOCAL_DATE, "DATETXT": tt.type.LOCAL_DATE},
    date_patterns={"DATE": "dd-MM-yyyy", "DATETXT": "dd-MM-yyyy"},
)

In [4]:
table.head(5)

,DATETXT,Sales
DATE,,
2022-11-01,2022-11-01,100.0
2023-11-01,2023-11-01,100.0
2022-12-01,2022-12-01,100.0
2023-12-01,2023-12-01,100.0
2022-01-01,2022-01-01,100.0


In [5]:
cube = session.create_cube(table, mode="auto")
l, m = cube.levels, cube.measures

In [6]:
cube.create_date_hierarchy(
    "Date Parts",
    column=table["DATE"],
    levels={"Year": "y", "Quarter": "QQQ", "Month": "MMMM", "Day": "d"},
)

In [7]:
h = cube.hierarchies
h["Date Parts"] = {**h["Date Parts"].levels, "DATE": table["DATE"]}

In [8]:
# Ensure the months are in order. 

l["Month"].order = tt.CustomOrder(
    first_elements=[
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
    ]
)

In [9]:
levels = ([l["Date Parts", "DATE"]],)
cube.query(
    m["Sales.SUM"],
    include_totals=True,
    levels=[l["Date Parts", "DATE"]],
)

Sales.SUM
Year  Quarter Month    Day DATE                
Total                                  2,400.00
2022                                   1,200.00
      Q1                                 300.00
              January                    100.00
                       1                 100.00
...                                         ...
2023  Q4      November 1                 100.00
                           2023-11-01    100.00
              December                   100.00
                       1                 100.00
                           2023-12-01    100.00

[83 rows x 1 columns]

In [10]:
# You can see the Date Filters in Excel.
session.port

50544